<a href="https://colab.research.google.com/github/srikanthgr/pyspark-python/blob/main/pyspark_banking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.3/spark-2.4.3-bin-hadoop2.6.tgz
!tar -xvf spark-2.4.3-bin-hadoop2.6.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.3-bin-hadoop2.6"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [3]:
!wget https://raw.githubusercontent.com/srikanthgr/pyspark-python/main/bank_prospects.csv

--2021-03-05 06:58:30--  https://raw.githubusercontent.com/srikanthgr/pyspark-python/main/bank_prospects.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 306 [text/plain]
Saving to: ‘bank_prospects.csv’

bank_prospects.csv  100%[===================>]     306  --.-KB/s    in 0s      

2021-03-05 06:58:31 (19.3 MB/s) - ‘bank_prospects.csv’ saved [306/306]



In [4]:
!ls

bank_prospects.csv  spark-2.4.3-bin-hadoop2.6
sample_data	    spark-2.4.3-bin-hadoop2.6.tgz


In [6]:
bankProspectsDF = spark.read.csv("bank_prospects.csv", header=True)
bankProspectsDF.show()

+----+------+------+-------+---------+
| Age|Salary|Gender|Country|Purchased|
+----+------+------+-------+---------+
|  18| 20000|  Male|Germany|        N|
|  19| 22000|Female| France|        N|
|  20| 24000|Female|England|        N|
|  21|  null|  Male|England|        N|
|  22| 50000|  Male| France|        Y|
|  23| 35000|Female|England|        N|
|  24|  null|  Male|Germany|        N|
|  25| 32000|Female| France|        Y|
|null| 35000|  Male|Germany|        N|
|  27| 37000|Female| France|        N|
|  27| 37000|Female|unknown|        N|
+----+------+------+-------+---------+



In [10]:
bankProspectsDF1 = bankProspectsDF.filter(bankProspectsDF['country'] != "unknown")
bankProspectsDF1.show()

+----+------+------+-------+---------+
| Age|Salary|Gender|Country|Purchased|
+----+------+------+-------+---------+
|  18| 20000|  Male|Germany|        N|
|  19| 22000|Female| France|        N|
|  20| 24000|Female|England|        N|
|  21|  null|  Male|England|        N|
|  22| 50000|  Male| France|        Y|
|  23| 35000|Female|England|        N|
|  24|  null|  Male|Germany|        N|
|  25| 32000|Female| France|        Y|
|null| 35000|  Male|Germany|        N|
|  27| 37000|Female| France|        N|
+----+------+------+-------+---------+



In [11]:
bankProspectsDF1.printSchema()

root
 |-- Age: string (nullable = true)
 |-- Salary: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Purchased: string (nullable = true)



In [15]:
from pyspark.sql.types import IntegerType, FloatType
bankProspectsDF2 = bankProspectsDF1.withColumn("Age", bankProspectsDF1["Age"].cast(IntegerType())).withColumn("Salary", bankProspectsDF1["Salary"].cast(FloatType()))

bankProspectsDF2.printSchema()

root
 |-- Age: integer (nullable = true)
 |-- Salary: float (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Purchased: string (nullable = true)



In [17]:
from pyspark.sql.functions import mean
mean_age_val = bankProspectsDF2.select(mean(bankProspectsDF2['age'])).collect()
type(mean_age_val)
mean_age = mean_age_val[0][0]
mean_age

22.11111111111111

In [18]:
mean_salary_val = bankProspectsDF2.select(mean(bankProspectsDF2['salary'])).collect()
mean_salary = mean_salary_val[0][0]
mean_salary

31875.0

In [23]:
bankProspectsDF2.show()
bankbankProspectsDF3 = bankProspectsDF2.na.fill(mean_age,["age"])
bankbankProspectsDF3.show()
bankbankProspectsDF4 = bankbankProspectsDF3.na.fill(mean_salary,["salary"])
bankbankProspectsDF4.show()
bankbankProspectsDF4.printSchema()

+----+-------+------+-------+---------+
| Age| Salary|Gender|Country|Purchased|
+----+-------+------+-------+---------+
|  18|20000.0|  Male|Germany|        N|
|  19|22000.0|Female| France|        N|
|  20|24000.0|Female|England|        N|
|  21|   null|  Male|England|        N|
|  22|50000.0|  Male| France|        Y|
|  23|35000.0|Female|England|        N|
|  24|   null|  Male|Germany|        N|
|  25|32000.0|Female| France|        Y|
|null|35000.0|  Male|Germany|        N|
|  27|37000.0|Female| France|        N|
+----+-------+------+-------+---------+

+---+-------+------+-------+---------+
|Age| Salary|Gender|Country|Purchased|
+---+-------+------+-------+---------+
| 18|20000.0|  Male|Germany|        N|
| 19|22000.0|Female| France|        N|
| 20|24000.0|Female|England|        N|
| 21|   null|  Male|England|        N|
| 22|50000.0|  Male| France|        Y|
| 23|35000.0|Female|England|        N|
| 24|   null|  Male|Germany|        N|
| 25|32000.0|Female| France|        Y|
| 22|35000

In [24]:
bankbankProspectsDF4.write.format("csv").save("bank_prospects_transformed")

In [26]:
!ls
!ls bank_prospects_transformed/

bank_prospects.csv	    spark-2.4.3-bin-hadoop2.6
bank_prospects_transformed  spark-2.4.3-bin-hadoop2.6.tgz
sample_data
part-00000-3f1ed220-297b-4b76-94cd-e1c3b5a4d81c-c000.csv  _SUCCESS
